# Model Training
## Summary
- Split dataset
- build model pipeline
  - define model hyperparameters
- fit and tune model with crossvalidation (GridsearchCV)
- evaluate metrics

### Setup Dependencies

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
import pickle

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

In [4]:
df = pd.read_csv('../data/analytical_base_table.csv')

### Split your dataset

In [5]:
y = df.status

X = df.drop('status', axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1234, stratify=df.status)

print(len(X_train), len(X_test), len(y_train), len(y_test))

11254 2814 11254 2814


### Build model pipelines

In [7]:
pipeline_dict = {
    'l1': make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', random_state=123)),
    'l2': make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', random_state=123)),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier(random_state=123)),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=123))
}

### Declare Hyperparameters to Tune

In [8]:
pipeline_dict['l1'].get_params()

{'memory': None,
 'steps': [('standardscaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('logisticregression',
   LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='warn',
             n_jobs=None, penalty='l1', random_state=123, solver='warn',
             tol=0.0001, verbose=0, warm_start=False))],
 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'logisticregression': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l1', random_state=123, solver='warn',
           tol=0.0001, verbose=0, warm_start=False),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'logisticregression__C': 1.0,
 'logisticregression__class_weight': None,
 'logisticregression__dual': False,
 'logisti

In [9]:
l1_hyperparameters = { 'logisticregression__C': np.linspace(1e-3, 1e3, 10)}
l2_hyperparameters = { 'logisticregression__C': np.linspace(1e-3, 1e3, 10)}

In [10]:
rf_hyperparameters = { 
    'randomforestclassifier__n_estimators':[100,200],
    'randomforestclassifier__max_features':['auto', 'sqrt', 0.33]
}

In [11]:
gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators': [100,200],
    'gradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2],
    'gradientboostingclassifier__max_depth': [1,3,5]
}

In [12]:
hyperparameter_dict = {
    'l1': l1_hyperparameters,
    'l2': l2_hyperparameters,
    'rf': rf_hyperparameters, 
    'gb': gb_hyperparameters
}

### Fit and Tune Models with Cross-Validation

In [ ]:
fitted_models = {}

for name, pipeline in pipeline_dict.items():
    model = GridSearchCV(pipeline, hyperparameter_dict[name], cv=10)
    model.fit(X_train, y_train)
    fitted_models[name] = model
    print(name, 'has been fitted')

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/deniel.horvatic/repository/machine-lea

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

...

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/mac

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/deniel.horvatic/repository/machine-learning-masterclass/project_03_employee_analytics_complete_model/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/deniel.horvatic/repository/machine-learning-ma

### Evaluate Metrics

In [ ]:
for name, model in fitted_models.items():
    print(name, model.best_score_)

### Area und ROC curve
Area under ROC curve is the most reliable metric for classification tasks

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
prediction = fitted_models['l1'].predict(X_test)

prediction[:10]


In [ ]:
# display confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, prediction))

In [ ]:
# predict probability for each class
prediction = fitted_models['l1'].predict_proba(X_test)

prediction = [p[1] for p in prediction]

prediction[:10]

In [ ]:
# calculate the ROC
fpr, tpr, thresholds = roc_curve(y_test, prediction)

In [ ]:
pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Thresholds': thresholds}).tail(10)

In [ ]:
def plot_roc_curve(fpr,tpr):
    # plot ROC curve
    figure = plt.figure(figsize=(8,8))
    plt.title('Receiver Operating Characteristic')

    plt.plot(fpr, tpr, label='l1')
    plt.legend(loc='lower right')

    plt.plot([0,1],[0,1], 'k--')

    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.ylabel('True Position Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, prediction)

print(auc(fpr,tpr))

In [ ]:
for name, model in fitted_models.items():
    prediction = model.predict_proba(X_test)
    prediction = [p[1] for p in prediction]
    
    fpr,tpr, thresholds = roc_curve(y_test, prediction)
    plot_roc_curve(fpr,tpr)
    

In [ ]:
# save winning pipeline
with open('../model/final_model.pk1', 'wb') as f:
    pickle.dump(fitted_models['rf'].best_estimator_, f)